<a href="https://colab.research.google.com/github/tillns/CIL_project/blob/master/Code/ownGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# TODO: 1) add tensorboard support to better analyze training 
# 2) a progress bar would be kinda nice as well
# 3) tweak params, of course
# 4) think about batch norm/instance norm
# 5) maybe add some normal conv layers to gen/dis
# 6) name files based on parameters and architecture, s.t. there are individual ones for different kinds of training
# 7) Adding noise to real input images before feeding them into the dis might help https://github.com/soumith/ganhacks/issues/14
# 8) save model every few minutes maybe
# 9) progressive growing proposed by Sven: https://github.com/tkarras/progressive_growing_of_gans --- Code at https://github.com/tkarras/progressive_growing_of_gans/blob/master/networks.py
#    not yet implemented use_wscale? weight decay 0.999 I think
import os, time, itertools, pickle
import numpy as np
import tensorflow as tf
from PIL import Image

# for now, only those images with label 1 are kept
# consider including also the fake ones

original_image_size = 1000
image_size = 64
patch_divide_factor = 4
factor_to_downsample = 1024//image_size//patch_divide_factor
downsample_size = original_image_size//factor_to_downsample  
# e.g. for training on 64x64, the original images may be downsampled to 250x250
# and during training. random 64x64 (appr. 1/4 = 1/patch_divide_factor) patches
# are drawn from these images
print("Downsampling input images to: {}x{}".format(downsample_size, downsample_size))
image_channels = 1
home_dir = os.path.expanduser("~")
image_directory = os.path.join(home_dir, "dataset/cil-cosmology-2018/cosmology_aux_data_170429/labeled")
label_path = os.path.join(home_dir, "dataset/cil-cosmology-2018/cosmology_aux_data_170429/labeled.csv")
print(label_path)

convs = []
lrelus = []
resblocks = []
upsamples = []
tf.reset_default_graph()
tf.InteractiveSession().close()

# params to tweak
gen_num_upsample = 2
gen_res = pow(2, gen_num_upsample)
gen_num_resblocks = 0
gen_num_normal_convs = 0
dis_num_downsample = 2
dis_res = pow(2, dis_num_downsample)
dis_num_normal_convs = 0
dis_num_resblocks = 0
# my guess is that b&w star pics won't need many feature maps, bc the only 
# features are white dots and black background
gen_start_features = 32
dis_start_features = 16
normal_conv_kernel_dim = 3
downsample_conv_kernel_dim = 4
lrelu_factor = 0.2
batch_size = 32
lr = 0.0002
train_epoch = 200

try:
  f=open(label_path,'r')
  label_list = []
  for line in f:
    if not "Id,Actual" in line:
      label_list.append(line.split(","))
  label_list = sorted(label_list)
  labels = np.zeros(len(label_list))
  for ind, label in enumerate(label_list):
    labels[ind] = label[1]

  imgs = np.empty((0, downsample_size, downsample_size, image_channels)) 
  img_list = []
  for filename in os.listdir(image_directory):
    if filename.endswith(".png") and not filename.startswith("._"):
       img_list.append(filename)


  img_list = sorted(img_list)
  for ind, filename in enumerate(img_list):
    if labels[ind] == 1:
      img = Image.open(os.path.join(image_directory, filename)).resize((downsample_size, downsample_size))
      imgs = np.append(imgs, np.array(img).reshape((1, downsample_size, downsample_size, image_channels)), axis=0)
  # imgs = (imgs - 0.5) / 0.5  # if they go from 0 to 1, use this to make them go from -1 to 1

  
except FileNotFoundError:
  print("Dataset not found. Using dummy dataset")
  labels = np.ones(1000)
  imgs = np.random.normal(0, 1, (1000, image_size, image_size, image_channels))
  

# this could get useful if we include the fake images from the dataset with label 0 
dataset_length = int(imgs.shape[0])
labels_wide = np.zeros((dataset_length, image_size//dis_res, image_size//dis_res, 1))
for ind in range(dataset_length):
  labels_wide[ind, :, :, :] = labels[ind]

#dataset = tf.data.Dataset.from_tensor_slices(imgs)



Downsampling input images to: 250x250
/home/tillns/dataset/cil-cosmology-2018/cosmology_aux_data_170429/labeled.csv


In [0]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
from random import randint

def lrelu(x, th=0.2):
    return tf.maximum(th * x, x)
  
def pixel_norm(x, epsilon=1e-8):
  return x * tf.rsqrt(tf.reduce_mean(tf.square(x), axis=1, keepdims=True) + epsilon)

# Minibatch standard deviation.

def minibatch_stddev_layer(x, group_size=4):
    with tf.variable_scope('MinibatchStddev'):
        group_size = tf.minimum(group_size, tf.shape(x)[0])     # Minibatch must be divisible by (or smaller than) group_size.
        s = x.shape                                             # [NCHW]  Input shape.
        y = tf.reshape(x, [group_size, -1, s[1], s[2], s[3]])   # [GMCHW] Split minibatch into M groups of size G.
        y = tf.cast(y, tf.float32)                              # [GMCHW] Cast to FP32.
        y -= tf.reduce_mean(y, axis=0, keepdims=True)           # [GMCHW] Subtract mean over group.
        y = tf.reduce_mean(tf.square(y), axis=0)                # [MCHW]  Calc variance over group.
        y = tf.sqrt(y + 1e-8)                                   # [MCHW]  Calc stddev over group.
        y = tf.reduce_mean(y, axis=[1,2,3], keepdims=True)      # [M111]  Take average over fmaps and pixels.
        y = tf.cast(y, x.dtype)                                 # [M111]  Cast back to original data type.
        y = tf.tile(y, [group_size, 1, s[2], s[3]])             # [N1HW]  Replicate over group and pixels.
        return tf.concat([x, y], axis=1)                        # [NCHW]  Append as new fmap.

def add_activated_norm(x, lrelu_factor=0.2, isTrain=True, normalization='pixel'):
  if normalization == 'pixel':
    x = pixelnorm(x)
  elif normalization == 'batch'
    x = tf.layers.batch_normalization(convs[-1], training=isTrain)

  lrelus.append(lrelu(x, lrelu_factor))
  

# number of input features MUST equal number of output features!
def get_resblock(x, features=256, kernel=[3, 3], strides=(1, 1), padding='same', lrelu_factor=0.2, isTrain=True):
    convs.append(tf.layers.conv2d(x, features, kernel, strides, padding))
    lrelus.append(lrelu(tf.layers.batch_normalization(convs[-1], training=isTrain), lrelu_factor))
    convs.append(tf.layers.conv2d(lrelus[-1], features, kernel, strides, padding))
    lrelus.append(lrelu(tf.layers.batch_normalization(convs[-1], training=isTrain), lrelu_factor))
    return lrelus[-1]+x;
  
  
# Conv blocks after growing GANs paper. In the paper, they describe their up-/downsampling
# methods to be separate from the conv while in the code, they also explore the combination
# with transposed/strided convolution. Maybe add this here, it's more efficient
def get_convblock(x, features, upsample=True, kernel=[3, 3], strides=(1, 1), padding='same', lrelu_factor=0.2, nomalization='pixel', isTrain=True):
  if upsample
    upsamples.append(tf.image.resize(x, (2*x.shape[1], 2*x.shape[2])), method=1)
  convs.append(tf.layers.conv2d(upsamples[-1], features, kernel, strides, padding))
  add_activated_norm(convs[-1], lrelu_factor, isTrain, normalization)
  convs.append(tf.layers.conv2d(lrelus[-1], features, kernel, strides, padding))
  add_activated_norm(convs[-1], lrelu_factor, isTrain, normalization)
  if not upsample:
    return tf.layers.average_pooling2d(lrelus[-1], 2, 2)  # not completely sure about second 2
  return lrelus[-1]
  
  
def generator_progr(x, isTrain=True, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
      
      features = gen_start_features
      # first conv block
      x = tf.reshape(x, (-1, features))  # this assumes the input to have 1x1 spatial res
      x = tf.layers.dense(pixel_norm(x), features*16)
      x = tf.reshape(x, (-1, features, 4, 4))
      convs.append(tf.layers.conv2d(x, features, kernel=[4, 4], stride=(1, 1), padding='same'))
      add_activated_norm(convs[-1], lrelu_factor, isTrain, normalization='pixel')
      convs.append(tf.layers.conv2d(lrelus[-1], features, kernel, stride=(1, 1), padding='same'))
      add_activated_norm(convs[-1], lrelu_factor, isTrain, normalization)
      x = lrelus[-1]
      for num_upsample in range(gen_num_upsample):
        if num_upsample >= 3: 
          features = features//2
        x = get_comvblock(x, features, upsample=True)
      
      # this layer has to be cross faided for res changes
      o = tf.layers.conv2d(x, image_channels, kernel=(1, 1), stides=(1, 1), padding='same')
      # they mention a linear activation, which probably means none at all...
      if False:  
        o = tf.nn.tanh(conv5)

      return o
    
# D(x)
def discriminator_progr(x, isTrain=True, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
      
      features = dis_start_features
      # downsampling convs (reduce resolution, increse features)
      for num_downsample in range(dis_num_downsample):
        if num_downsample == 0:
            input_im = x
        else:
          input_im = lrelus[-1]
        convs.append(tf.layers.conv2d(input_im, features, [downsample_conv_kernel_dim, downsample_conv_kernel_dim], strides=(2, 2), padding='same'))
        # maybe change to instance normalization
        lrelus.append(lrelu(tf.layers.batch_normalization(convs[-1], training=isTrain), lrelu_factor))
        features = features*2
        
      # resblocks (features kept constant)  
      for num_resblock in range(dis_num_resblocks):
        if num_resblock == 0:
          if dis_num_downsample == 0:
            input_im = x
          else:
            input_im = lrelus[-1]
            features=features//2
        else:
          input_im = resblocks[-1]
        resblocks.append(get_resblock(input_im, features=features, kernel = [normal_conv_kernel_dim, normal_conv_kernel_dim], lrelu_factor=lrelu_factor, isTrain=isTrain))

      # output layer without fc, s.t. local patches are rated and generator is
      # forced to be cinsistent everywhere (PatchGAN). Could be changed tho...
      if dis_num_resblocks == 0:
        if dis_num_downsample == 0:
          input_im = x
        else:
          input_im = lrelus[-1]
      else:
        input_im = resblocks[-1]
      conv6 = tf.layers.conv2d(input_im, 1, [normal_conv_kernel_dim, normal_conv_kernel_dim], strides=(1, 1), padding='same')
      #fc = tf.layers.dense(conv6, 32*32, activation='sigmoid')
      o = tf.nn.sigmoid(conv6)

      return o
  
      
    
# G(z)
def generator(x, isTrain=True, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
      
      features = gen_start_features
      # resblocks (num features kept constant)
      for num_resblock in range(gen_num_resblocks):
          if num_resblock == 0:
            input_im = x
          else:
            input_im = resblocks[-1]
          resblocks.append(get_resblock(input_im, features=features, kernel = [normal_conv_kernel_dim, normal_conv_kernel_dim], lrelu_factor=lrelu_factor, isTrain=isTrain))

      # upsample + conv (num features reduced)
      for num_upsample in range(gen_num_upsample):
        if num_upsample == 0:
          if gen_num_resblocks == 0:
            input_im = x
          else:
            input_im = resblocks[-1]
        else:
          input_im = lrelus[-1]
        upsamples.append(tf.image.resize_images(input_im, (2*int(input_im.shape[1]), 2*int(input_im.shape[2])), method=1))
        convs.append(tf.layers.conv2d(upsamples[-1], features, [normal_conv_kernel_dim, normal_conv_kernel_dim], strides=(1, 1), padding='same'))
        # maybe change to instance normalization (tf.contrib.layers.instance_norm(input, trainable)) 
        # this does not exist in tf 1.3 yet and I haven't gotten it to work on 1.13 either
        # error probably lies in trainable, haven't understood that yet
        lrelus.append(lrelu(tf.layers.batch_normalization(convs[-1], training=isTrain), lrelu_factor))
        features = features//2

      # output layer
      if gen_num_upsample == 0:
        if gen_num_resblocks == 0:
          input_im = x
        else:
          input_im = resblocks[-1]
      else:
        input_im = lrelus[-1]
      conv5 = tf.layers.conv2d(lrelus[-1], 1, [normal_conv_kernel_dim, normal_conv_kernel_dim], strides=(1, 1), padding='same')
      # never normalize output
      o = tf.nn.tanh(conv5)

      return o

# D(x)
def discriminator(x, isTrain=True, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
      
      features = dis_start_features
      # downsampling convs (reduce resolution, increse features)
      for num_downsample in range(dis_num_downsample):
        if num_downsample == 0:
            input_im = x
        else:
          input_im = lrelus[-1]
        convs.append(tf.layers.conv2d(input_im, features, [downsample_conv_kernel_dim, downsample_conv_kernel_dim], strides=(2, 2), padding='same'))
        # maybe change to instance normalization
        lrelus.append(lrelu(tf.layers.batch_normalization(convs[-1], training=isTrain), lrelu_factor))
        features = features*2
        
      # resblocks (features kept constant)  
      for num_resblock in range(dis_num_resblocks):
        if num_resblock == 0:
          if dis_num_downsample == 0:
            input_im = x
          else:
            input_im = lrelus[-1]
            features=features//2
        else:
          input_im = resblocks[-1]
        resblocks.append(get_resblock(input_im, features=features, kernel = [normal_conv_kernel_dim, normal_conv_kernel_dim], lrelu_factor=lrelu_factor, isTrain=isTrain))

      # output layer without fc, s.t. local patches are rated and generator is
      # forced to be cinsistent everywhere (PatchGAN). Could be changed tho...
      if dis_num_resblocks == 0:
        if dis_num_downsample == 0:
          input_im = x
        else:
          input_im = lrelus[-1]
      else:
        input_im = resblocks[-1]
      conv6 = tf.layers.conv2d(input_im, 1, [normal_conv_kernel_dim, normal_conv_kernel_dim], strides=(1, 1), padding='same')
      #fc = tf.layers.dense(conv6, 32*32, activation='sigmoid')
      o = tf.nn.sigmoid(conv6)

      return o

size_figure_grid = 5
fixed_z_ = np.random.normal(0, 1, (size_figure_grid*size_figure_grid, image_size//gen_res, image_size//gen_res, gen_start_features))
def show_result(num_epoch, show = False, save = False, path = 'result.png'):
    test_images = sess.run(G_z, {z: fixed_z_, isTrain: False})

    fig_size = image_size*size_figure_grid*1.3/100  # appr. formula s.t. image sizes aren't reduced
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(fig_size, fig_size))
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)

    for k in range(size_figure_grid*size_figure_grid):
        i = k // size_figure_grid
        j = k % size_figure_grid
        ax[i, j].cla()
        ax[i, j].imshow(np.reshape(test_images[k], (image_size, image_size)), cmap='gray')

    label = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, label, ha='center')

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

# plots graph with dis and gen loss 
def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    x = range(len(hist['D_losses']))

    y1 = hist['D_losses']
    y2 = hist['G_losses']

    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

        
#with tf.device('/cpu:0'):
with tf.device('/device:GPU:0'): 
# variables : input
  x = tf.placeholder(tf.float32, shape=(None, image_size, image_size, image_channels))
  z = tf.placeholder(tf.float32, shape=(None, image_size//gen_res, image_size//gen_res, gen_start_features))
  isTrain = tf.placeholder(dtype=tf.bool)

  # networks : generator
  G_z = generator(z, isTrain)

  # networks : discriminator
  D_real_logits = discriminator(x, isTrain)
  D_fake_logits = discriminator(G_z, isTrain, reuse=True)

  # loss for each network
  # think about these losses, there are many variants (vanilla GAN, WGAN, ...)
  D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones([batch_size, image_size//dis_res, image_size//dis_res, 1])))
  D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros([batch_size, image_size//dis_res, image_size//dis_res, 1])))
  D_loss = D_loss_real + D_loss_fake
  G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones([batch_size, image_size//dis_res, image_size//dis_res, 1])))

  # trainable variables for each network
  T_vars = tf.trainable_variables()
  D_vars = [var for var in T_vars if var.name.startswith('discriminator')]
  G_vars = [var for var in T_vars if var.name.startswith('generator')]

  # optimizer for each network
  # This should probably not be experimented with. ADAM is standard nowadays.
  with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
      D_optim = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(D_loss, var_list=D_vars)
      G_optim = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(G_loss, var_list=G_vars)

  # open session and initialize all variables
  sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=False, log_device_placement=True))
  tf.global_variables_initializer().run()

  # resize and normalization
  # train_set = dataset#.eval()
  #train_set = (train_set - 0.5) / 0.5  # normalization; range: -1 ~ 1

  # results save folder
  root = 'DCGAN_results/'
  model = 'DCGAN_'
  if not os.path.isdir(root):
      os.mkdir(root)
  if not os.path.isdir(root + 'Fixed_results'):
      os.mkdir(root + 'Fixed_results')

  train_hist = {}
  train_hist['D_losses'] = []
  train_hist['G_losses'] = []
  train_hist['per_epoch_ptimes'] = []
  train_hist['total_ptime'] = []

  # training-loop
  np.random.seed(int(time.time()))
  print('training start!')
  start_time = time.time()
  for epoch in range(train_epoch):
    G_losses = []
    D_losses = []
    epoch_start_time = time.time()
    #dataset.shuffle(buffer_size=dataset_length)
    #dataset = dataset.batch(batch_size)
    #iterator = dataset.make_one_shot_iterator()

    # TODO: include progress bar
    # to avoid overfitting, the images are shuffled every epoch
    # if fake images are included, suffle an index list to access the corresponding label along with the image
    np.random.shuffle(imgs)
    for iteration in range(dataset_length // batch_size):
      #x_ = iterator.get_next()
      #x_ = x_.eval()

      # update discriminator
      x_ = imgs[iteration*batch_size:(iteration+1)*batch_size]
      random_startx = randint(0, downsample_size-image_size)
      random_starty = randint(0, downsample_size-image_size)
      # draw a random patch from the input
      x_ = x_[:, random_startx:random_startx+image_size, random_starty:random_starty+image_size]
      for i in range(2):
        if randint(0, 1):
          x_ = np.flip(x_, axis=i+1)  # randomly flip x- and y-axis
      z_ = np.random.normal(0, 1, (batch_size, image_size//gen_res, image_size//gen_res, gen_start_features))

      loss_d_, _ = sess.run([D_loss, D_optim], {x: x_, z: z_, isTrain: True})
      D_losses.append(loss_d_)

      # update generator
      z_ = np.random.normal(0, 1, (batch_size, image_size//gen_res, image_size//gen_res, gen_start_features))
      loss_g_, _ = sess.run([G_loss, G_optim], {z: z_, x: x_, isTrain: True})
      G_losses.append(loss_g_)

    epoch_end_time = time.time()
    per_epoch_ptime = epoch_end_time - epoch_start_time
    print('[%d/%d] - ptime: %.2f loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), train_epoch, per_epoch_ptime, np.mean(D_losses), np.mean(G_losses)))
    fixed_p = root + 'Fixed_results/' + model + str(epoch + 1) + '.png'
    show_result((epoch + 1), save=True, show=False, path=fixed_p)
    train_hist['D_losses'].append(np.mean(D_losses))
    train_hist['G_losses'].append(np.mean(G_losses))
    train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

  end_time = time.time()
  total_ptime = end_time - start_time
  train_hist['total_ptime'].append(total_ptime)

  print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(train_hist['per_epoch_ptimes']), train_epoch, total_ptime))
  print("Training finish!... save training results")
  with open(root + model + 'train_hist.pkl', 'wb') as f:
      pickle.dump(train_hist, f)

  show_train_hist(train_hist, save=True, path=root + model + 'train_hist.png')

  # this thing makes a gif out of the results after every epoch, so it's not
  # necessary 
  try:
    import imageio
    images = []
    for e in range(train_epoch):
        img_name = root + 'Fixed_results/' + model + str(e + 1) + '.png'
        images.append(imageio.imread(img_name))
    imageio.mimsave(root + model + 'generation_animation.gif', images, fps=5)
  except Exception as e:
    print("Haven't gotten imageio to work on the cluster, smh. Error message:")
    print(e)


Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
training start!
[1/200] - ptime: 2.10 loss_d: 1.410, loss_g: 0.499
[2/200] - ptime: 0.83 loss_d: 1.328, loss_g: 0.549
[3/200] - ptime: 0.89 loss_d: 1.279, loss_g: 0.587
[4/200] - ptime: 0.88 loss_d: 1.256, loss_g: 0.608
[5/200] - ptime: 0.91 loss_d: 1.238, loss_g: 0.622
[6/200] - ptime: 0.85 loss_d: 1.217, loss_g: 0.631
[7/200] - ptime: 0.88 loss_d: 1.205, loss_g: 0.637
[8/200] - ptime: 0.88 loss_d: 1.190, loss_g: 0.641
[9/200] - ptime: 0.87 loss_d: 1.177, loss_g: 0.645
[10/200] - ptime: 0.91 loss_d: 1.162, loss_g: 0.648
[11/200] - ptime: 0.88 loss_d: 1.149, loss_g: 0.651
[12/200] - ptime: 0.87 loss_d: 1.139, loss_g: 0.653
[13/200] - ptime: 0.87 loss_d: 1.128, loss_g: 0.656
[14/200] - ptime: 0.89 loss_d: 1.119, loss_g: 0.659
[15/200] - ptime: 0.87 loss_d: 1.110, loss_g: 0.661
[16/200]

*# Load Dataset*

In [0]:
tf.reset_default_graph()

In [0]:
sess.close()